In [1]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time

get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
import pickle

#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    data = []
    for res in JSONRes['candles']:
        temp = [((datetime.datetime.fromisoformat(res['time'][:19]))+datetime.timedelta(hours=+9)).strftime('%Y-%m-%d %H:%M')]#indexとなる時間を追加
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data




def get_now_data(time, period_Days, money='USD_JPY', foot = 'D', back_days = 0):
    all_data = []
    
    time = time.replace(hour=7, minute=0, second=0, microsecond=0)
    #入力された日の当日の7時
    
    NY = datetime.timedelta(hours=-14)
    back = datetime.timedelta(days=-back_days)
    
    date_to = time + NY + datetime.timedelta(seconds=0) + back
    date_from = date_to + datetime.timedelta(days=-period_Days)
    
    ret = getCandleDataFromOanda(money, api, date_from, date_to, foot)
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加

    # pandas DataFrameへ変換
    df = pd.DataFrame(all_data)

    df.columns = ['Datetime','Volume', 'Open', 'High', 'Low', 'Close']
    df['Open'] = df['Open'].astype('double')
    df['Close'] = df['Close'].astype('double')
    
    df['Rate']=(df['Close']-df['Open'])/df['Open']
    
    df = df.set_index('Datetime')

    return df
    
def get_csv(back_days=100, days=3650, money='USD_JPY'):
    time = datetime.datetime.now()
    #df = get_now_data(time, period_Days=days, money=money, foot = 'D', back_days=back_days)
    df = get_now_data(time, period_Days=days, money=money, foot = 'H1', back_days=back_days)
    date_to = (time+datetime.timedelta(days=-back_days)+datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
    date_from = (time + datetime.timedelta(days=-days)+datetime.timedelta(days=-back_days)).strftime("%Y-%m-%d")
    df.to_csv(str(money)+"_"+ date_from + "_" + date_to +'.csv', mode = 'w')
    print('"'+str(money)+"_"+ date_from + "_" + date_to + '.csv', end='", ')
    
    return df

def get_moneys_csv(back_days=100, days=3650):
    moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY']
    for i in range (len(moneylist)):
        get_csv(back_days, days, money=moneylist[i])
        
# def classify(x, c, R=0.001):
#         if x<=-R:
#             c[0]+=1
#             return 0
#         elif -R<x<R:
#             c[1]+=1
#             return 1
#         elif R<=x:
#             c[2]+=1
#             return 2
        
def classify_two(x, c):
        if x<=0:
            c[0]+=1
            return -1
        elif 0<x:
            c[1]+=1
            return 1

def predict(dataname, moneyname, days):

    df = pd.read_csv(dataname, index_col='Datetime')
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df['class'] = (df["Rate"]).apply(lambda x: classify_two(x, count))
    df['class'] = df['class'].shift(-1)
    print("Train Data")
    print(" LOW/HIGH")
    print(count)
    
    X = df.drop('class', axis=1)
    y = df['class']
    X = X.drop(X.index[len(X)-1])
    y = y.dropna()
    
    #df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = 'D', back_days = 100)
    df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = 'H1', back_days = 100)
    print("money: "+moneyname)
    count = [0,0]
    #df['class'] = (df["Rate"]).apply(lambda x: classify(x, count, 0.001))
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)
    
    print("learning data")
    print(df.head(1).index.values, end=' to ')
    print(df.tail(1).index.values)
    
    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    X_train = X
    y_train = y
    X_test = X_now
    y_test = y_now
    
    
    
    
    names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"]
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        DecisionTreeClassifier(max_depth=10),
        RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
        AdaBoostClassifier(),
        GaussianNB(),
        LinearDiscriminantAnalysis(),
        QuadraticDiscriminantAnalysis()]
    
    accuracy_scores = []
    precision_scores = []
    next_class = []
    profit = []
    
    
    for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        result = clf.predict(X_test)
        next_class.extend(clf.predict(X_next))
        
        print('accuracy_score: ', end='')
        print(accuracy_score(y_test, result))
        accuracy_scores.extend([accuracy_score(y_test, result)])
        print('confusion_matrix: ')
        print(confusion_matrix(y_test, result))
        print('precision_score: ', end='')
        print(precision_score(y_test, result, average=None))
        precision_scores.extend([precision_score(y_test, result, average=None)])
        
        possestion = 50000
        pay = possestion*0.1
        po_rate = 1.95
        
        win = 0
        lose = 0
        
        for i in range(len(result)):
            possestion -= pay
            if (result[i]==-1):#Low予想のとき
                if (result[i]==y_test[i]): #予測成功
                    possestion += pay*po_rate
                    win+=1
                else:
                    lose+=1

            elif(result[i]==1):#High予想の時
                if (result[i]==y_test[i]): #予測成功
                    possestion += pay*po_rate
                    win+=1
                else:
                    lose+=1
            else:  #Mid予想の時
                possestion += pay
        print(str(len(result))+"days trade")
        print("posestion: "+ str(possestion))
        print("profit: "+str(win*0.95-lose))
        
        print('')
        
        profit.extend([win*0.95-lose])
        
        precision = precision_score(y_test, result, average=None)
        if((win*0.95-lose)>1 and precision[0]!=0 and precision[1]!=0):
            model_name = moneyname + "_" + name + ".sav"
            print("CLEAR: " + model_name)
            pickle.dump(clf, open(model_name, 'wb'))
        
    
    return accuracy_scores, precision_scores, next_class, profit

def make_models(times, datas):
    namelist = datas
    moneylist = ['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average']
    
    df = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_pre = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_next = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    df_profit = pd.DataFrame(index=['USD_JPY', 'EUR_JPY', 'AUD_JPY', 'GBP_JPY', 'NZD_JPY', 'CAD_JPY', 'CHF_JPY', 'ZAR_JPY', 'Average'], columns = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Decision Tree",
             "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
             "Quadratic Discriminant Analysis"])
    
    for i in range(len(namelist)):
        print(namelist[i])
        acc, pre, next_class, profit = predict(namelist[i], moneylist[i], days = 60)
        print(acc)
        df.loc[moneylist[i]]=acc
        df_pre.loc[moneylist[i]]=pre
        df_next.loc[moneylist[i]]=next_class
        df_profit.loc[moneylist[i]]=profit
        
    
    df.loc[moneylist[i+1]] = df.mean()
    df_pre.loc[moneylist[i+1]] = df_pre.mean()
    df_profit.loc[moneylist[i+1]] = df_profit.mean()
    print(df)
    print(df_pre)
    print(df_next)
    print(df_profit)
    df.to_csv('accuracy_result_'+str(times)+'.csv', mode = 'w')
    df_pre.to_csv('precision_result_'+str(times)+'.csv', mode = 'w')
    df_next.to_csv('predict_result_'+str(times)+'.csv', mode = 'w')
    df_profit.to_csv('profit_result_'+str(times)+'.csv', mode = 'w')
  

def calc_profit(predict_result, true_data):
    win=0
    lose=0
    for i in range(len(predict_result)):
            if (predict_result[i]==-1):#Low予想のとき
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
            elif(predict_result[i]==1):#High予想の時
                if (predict_result[i]==true_data[i]): #予測成功
                    win+=1
                else:
                    lose+=1
    profit = win*0.95-lose #利率を算出
    values = predict_result*profit #予測結果×利率により予測の価値を算出
    return profit, values

def plot_result(index, predict_result, true_data):
    plt.figure(figsize=(20,5))
    index = index.astype(float)

    color = []
    marker = []
    for x in range(0,len(predict_result)):
        if true_data.values[x] == predict_result[x]:
            color.extend(['blue'])

        else:
            color.extend(['red'])


    plt.scatter(index.index.values, index["Rate"].values, color=color)

    plt.xticks(rotation=90)

def ensemble(model_names, moneyname, days):
    
    df_now = get_now_data(datetime.datetime.now(), period_Days=days, money=moneyname, foot = 'D', back_days = 0)
    print("money: "+moneyname)
    count = [0,0]
    
    df_now['class'] = (df_now["Rate"]).apply(lambda x: classify_two(x, count))
    df_now['class'] = df_now['class'].shift(-1)
    print("NOW Data from" +str(days)+ "days ago")
    print(" LOW/MID/HIGH")
    print(count)

    print("")
    print("test data")
    print(df_now.head(1).index.values,end=' to ')
    print(df_now.tail(1).index.values)
    print("")
    X_now = df_now.drop('class', axis=1)
    
    X_next = X_now.tail(1) #最終予測に利用
    y_now = df_now['class']
    X_now = X_now.drop(X_now.index[len(X_now)-1])
    y_now = y_now.dropna()
    
    values = []
    values_sum = []
    win=0
    lose=0
    answer = 0
    
    for x in range(len(model_names)):
        clf = pickle.load(open(model_names[x], 'rb')) #clfをload
        print(model_names[x])
        result = clf.predict(X_now) #predict
         
        #score表示
        print("accuracy_score: ", end="")
        print(accuracy_score(result, y_now))
        print("precision_score: ", end="")
        print(precision_score(result, y_now, average=None))
        print(confusion_matrix(result, y_now))
        
        profit, values = calc_profit(result, y_now) #予測結果について利率と、予測*利率による予測の信頼性を算出
        
        answer += clf.predict(X_next)[0]*profit
        
        if len(values_sum)==0:#1回目は配列をコピー
            values_sum = values
        else:
            values_sum += values #２回目からは加算
        print(values)
        
    
        #plot_result(X_now, result, y_now)
        
    ensemble_result = np.where(values_sum<0, -1, 1)
    
    print("")
    print("ensemble_result: ",end='')
    print(accuracy_score(ensemble_result, y_now.values))
    print(ensemble_result)
    print(y_now.values)
    print(values_sum)
    print("precision_score: ", end="")
    print(precision_score(result, y_now, average=None))
    print("profit: ",end='')
    profit, values = calc_profit(ensemble_result, y_now)
    print(profit)
    print("next: ",end='')
    print(answer)
    print(X_next.head(1))
    plot_result(X_now, ensemble_result, y_now)
        
    
    
    
    
def main():
    models = ["GBP_JPY_AdaBoost.sav", "GBP_JPY_Nearest Neighbors.sav", "GBP_JPY_Quadratic Discriminant Analysis.sav", "CHF_JPY_Random Forest.sav"]
    ensemble(models, "GBP_JPY", 30)
#get_moneys_csv(back_days=300, days=120) # 指定期間のcsvをsave

datas = ["USD_JPY_2019-09-14_2020-01-11.csv", "EUR_JPY_2019-09-14_2020-01-11.csv", "AUD_JPY_2019-09-14_2020-01-11.csv", "GBP_JPY_2019-09-14_2020-01-11.csv", "NZD_JPY_2019-09-14_2020-01-11.csv", "CAD_JPY_2019-09-14_2020-01-11.csv", "CHF_JPY_2019-09-14_2020-01-11.csv", "ZAR_JPY_2019-09-14_2020-01-11.csv"]
make_models(0, datas)

# learning data
# ['2016-09-29 06:00'] to ['2020-01-10 07:00']

# test data
# ['2020-06-01 06:00'] to ['2020-07-29 06:00']

#main()

USD_JPY_2019-09-14_2020-01-11.csv
Train Data
 LOW/HIGH
[986, 1006]
money: USD_JPY
NOW Data from60days ago
 LOW/MID/HIGH
[554, 477]
learning data
['2019-09-14 06:00'] to ['2020-01-11 06:00']

test data
['2020-06-01 07:00'] to ['2020-07-30 05:00']

Nearest Neighbors
accuracy_score: 0.5009708737864078
confusion_matrix: 
[[292 261]
 [253 224]]
precision_score: [0.53577982 0.46185567]
1030days trade
posestion: -69000.0
profit: -23.80000000000001

Linear SVM
accuracy_score: 0.4640776699029126
confusion_matrix: 
[[ 54 499]
 [ 53 424]]
precision_score: [0.5046729  0.45937161]
1030days trade
posestion: -439500.0
profit: -97.90000000000003

RBF SVM
accuracy_score: 0.46601941747572817
confusion_matrix: 
[[ 26 527]
 [ 23 454]]
precision_score: [0.53061224 0.46279307]
1030days trade
posestion: -420000.0
profit: -94.0

Decision Tree
accuracy_score: 0.4796116504854369
confusion_matrix: 
[[353 200]
 [336 141]]
precision_score: [0.51233672 0.41348974]
1030days trade
posestion: -283500.0
profit: -66.700

accuracy_score: 0.5
confusion_matrix: 
[[305 188]
 [327 210]]
precision_score: [0.48259494 0.52763819]
1030days trade
posestion: -78750.0
profit: -25.75

Naive Bayes
accuracy_score: 0.5407766990291262
confusion_matrix: 
[[ 84 409]
 [ 64 473]]
precision_score: [0.56756757 0.53628118]
1030days trade
posestion: 330750.0
profit: 56.14999999999998

CLEAR: GBP_JPY_Naive Bayes.sav
Linear Discriminant Analysis
accuracy_score: 0.516504854368932
confusion_matrix: 
[[209 284]
 [214 323]]
precision_score: [0.49408983 0.53212521]
1030days trade
posestion: 87000.0
profit: 7.399999999999977

CLEAR: GBP_JPY_Linear Discriminant Analysis.sav
Quadratic Discriminant Analysis
accuracy_score: 0.5058252427184466
confusion_matrix: 
[[369 124]
 [385 152]]
precision_score: [0.48938992 0.55072464]
1030days trade
posestion: -20250.0
profit: -14.050000000000011

[0.49514563106796117, 0.5203883495145631, 0.5203883495145631, 0.5174757281553398, 0.4961165048543689, 0.5, 0.5407766990291262, 0.516504854368932, 0.505825

C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score: 0.5097087378640777
confusion_matrix: 
[[  0 502]
 [  3 525]]
precision_score: [0.         0.51119766]
1030days trade
posestion: 18750.0
profit: -6.25

Decision Tree
accuracy_score: 0.5194174757281553
confusion_matrix: 
[[348 154]
 [341 187]]
precision_score: [0.50507983 0.5483871 ]
1030days trade
posestion: 116250.0
profit: 13.25

CLEAR: CHF_JPY_Decision Tree.sav
Random Forest
accuracy_score: 0.5067961165048543
confusion_matrix: 
[[480  22]
 [486  42]]
precision_score: [0.49689441 0.65625   ]
1030days trade
posestion: -10500.0
profit: -12.100000000000023

AdaBoost
accuracy_score: 0.5029126213592233
confusion_matrix: 
[[480  22]
 [490  38]]
precision_score: [0.49484536 0.63333333]
1030days trade
posestion: -49500.0
profit: -19.900000000000034

Naive Bayes
accuracy_score: 0.512621359223301
confusion_matrix: 
[[  0 502]
 [  0 528]]
precision_score: [0.         0.51262136]
1030days trade
posestion: 48000.0
profit: -0.4000000000000341

Linear Discriminant Analysis
accuracy_s

C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


money: ZAR_JPY
NOW Data from60days ago
 LOW/MID/HIGH
[521, 510]
learning data
['2019-09-16 06:00'] to ['2020-01-11 06:00']

test data
['2020-06-01 07:00'] to ['2020-07-30 05:00']

Nearest Neighbors
accuracy_score: 0.5077669902912622
confusion_matrix: 
[[280 240]
 [267 243]]
precision_score: [0.511883   0.50310559]
1030days trade
posestion: -750.0
profit: -10.150000000000034

Linear SVM
accuracy_score: 0.5048543689320388
confusion_matrix: 
[[520   0]
 [510   0]]
precision_score: [0.50485437 0.        ]
1030days trade
posestion: -30000.0
profit: -16.0

RBF SVM


C:\Users\stana\.conda\envs\predict_rate\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score: 0.503883495145631
confusion_matrix: 
[[518   2]
 [509   1]]
precision_score: [0.50438169 0.33333333]
1030days trade
posestion: -39750.0
profit: -17.950000000000045

Decision Tree
accuracy_score: 0.49514563106796117
confusion_matrix: 
[[283 237]
 [283 227]]
precision_score: [0.5        0.48922414]
1030days trade
posestion: -127500.0
profit: -35.5

Random Forest
accuracy_score: 0.5135922330097087
confusion_matrix: 
[[ 83 437]
 [ 64 446]]
precision_score: [0.56462585 0.50509626]
1030days trade
posestion: 57750.0
profit: 1.5499999999999545

CLEAR: ZAR_JPY_Random Forest.sav
AdaBoost
accuracy_score: 0.4941747572815534
confusion_matrix: 
[[  2 518]
 [  3 507]]
precision_score: [0.4        0.49463415]
1030days trade
posestion: -137250.0
profit: -37.450000000000045

Naive Bayes
accuracy_score: 0.4912621359223301
confusion_matrix: 
[[  3 517]
 [  7 503]]
precision_score: [0.3        0.49313725]
1030days trade
posestion: -166500.0
profit: -43.30000000000001

Linear Discriminant An